In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('orange_small_churn_data.train', delimiter =',')
data['labels'] = pd.read_csv('orange_small_churn_labels.train', header=None)
#конвертируем колонку labels в int
data = data.astype({'labels': 'int32'})
data.head()


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,labels
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [3]:
from sklearn.model_selection import train_test_split
data, holdOut = train_test_split(data, test_size=0.15, random_state=42)

In [4]:
#на этой выборке будем обучать модели. переменная holdOut - это hold out, ее мы не будем использовать ни для обучения 
#ни для оценки baseline моделей
data.head()
#ограничим данные, т.к. иначе не хватает памяти
data = data.iloc[0:7000,:]

In [5]:
#отделим столбец с целевой переменной
target = np.array(data.iloc[:,-1])
#target = target[:,np.newaxis]
target.shape

(7000,)

In [6]:
#подготовим даные:
#выделим категориальные признаки (для baseline решения, возможно будет достаточно числовых)
numericalVarCount = 190
categorialVarCount = 40
data_num = data.iloc[:, 0:numericalVarCount]
#удалим числовые признаки, содержащие слишком большое количество NaN - значений
threshold = 0.7
NaN_frac = data_num.isna().sum(axis = 0)/data_num.shape[0]
data_num = data_num.loc[:,list(NaN_frac[NaN_frac < threshold].index)]
#Перед построением моделей, подготовим данные: заменим NaN на медианные значения,
data_num.fillna(data_num.median(), inplace=True)
#выполним стандартизацию числовых признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_num = scaler.fit_transform(data_num)
data_num.shape

/home/pavel/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/pavel/anaconda2/lib/python2.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(7000, 42)

In [7]:
#обработаем категориальные признаки методикой one-hot-encoding
from sklearn.feature_extraction import DictVectorizer as DV
data_cat = data.iloc[:, numericalVarCount:-1]
data_cat = data_cat.fillna('NA').astype(str)
NaN_frac = data_cat.isna().sum(axis = 0)/data_cat.shape[0]
NaN_frac
threshold = 0.01
NaN_frac = data_cat.isna().sum(axis = 0)/data_num.shape[0]
data_cat = data_cat.loc[:,list(NaN_frac[NaN_frac < threshold].index)]
encoder = DV(sparse = False)
data_cat_oh = encoder.fit_transform(data_cat.T.to_dict().values())
data_cat_oh

array([[1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [8]:
#объединяем обработанные числовые и категориальные признаки
data_all = np.hstack((data_num,data_cat_oh))
data_all

array([[-0.1168248 ,  0.04731182, -0.45785848, ...,  0.        ,
         0.        ,  1.        ],
       [-0.20777515,  0.04731182, -0.43908017, ...,  0.        ,
         0.        ,  1.        ],
       [-0.19997941,  0.04731182, -0.44220989, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.0180787 ,  0.04731182, -0.37492096, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.86589291,  0.04731182,  0.53113232, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47542905, -1.13972305, -0.4609882 , ...,  0.        ,
         0.        ,  1.        ]])

In [9]:
#будем использовать следующие модели для baseline решения:
#RandomForestClassifier, RidgeClassifier и SGDClassifier
#ввиду несбалансированности выборок везде используем class_weight='balanced'
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

In [10]:
from sklearn.model_selection import cross_val_score

clf = SGDClassifier(class_weight='balanced', max_iter=1000, tol=1e-4, alpha=0.1)
scores = cross_val_score(clf, data_all, target, cv=3, scoring = 'f1')
scores

array([0.18080149, 0.18094476, 0.18541667])

In [11]:
clf = RandomForestClassifier(n_estimators=10, max_depth=5, class_weight ='balanced')
scores = cross_val_score(clf, data_all, target, cv=3, scoring = 'f1')
scores

array([0.14348463, 0.06666667, 0.13253012])

In [12]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier(class_weight='balanced')
scores = cross_val_score(clf, data_all[0:5000,:], target[0:5000], cv=3, scoring = 'f1')
scores

array([0.10377358, 0.09389671, 0.07608696])